In [ ]:
#yt
import yt_dlp
from yt_dlp import YoutubeDL
from yt_dlp.utils import DownloadError
import logging
# Configure logging
logging.basicConfig(level=logging.INFO)
proxy = 'http://127.0.0.1:10808'
yt = YoutubeDL({
    'proxy': proxy,
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'noplaylist': True,
    'quiet': False,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
})
with open('./youtube_urls.txt', 'r') as f:
    urls = f.readlines()
for url in urls:
    url = url.strip()
    if not url:
        continue
    try:
        logging.info(f"Downloading {url}")
        yt.download([url])
    except DownloadError as e:
        logging.error(f"Failed to download {url}: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred while downloading {url}: {e}")
    else:
        logging.info(f"Successfully downloaded {url}")



In [ ]:
# api函数
import requests
import json
import time
import os
from typing import Optional, Dict
from tqdm import tqdm

def download_song(url: str, format:str = 'mp3',download_path: str='download', proxies: Optional[Dict[str, str]] = None, max_retries: int = 30, retry_delay: int = 1) -> Optional[str]:
    """
    从指定URL下载歌曲（MP3格式）。

    Args:
        url (str): 歌曲的原始URL。
        download_path (str): 歌曲的下载保存路径。
        proxies (dict, optional): 代理设置。默认为 None。
        max_retries (int): 获取下载链接的最大重试次数。默认为 30。
        retry_delay (int): 每次重试之间的等待时间（秒）。默认为 1。

    Returns:
        str or None: 如果下载成功，返回下载文件的完整路径；否则返回 None。
    """
    api_url = 'https://p.oceansaver.in/ajax/download.php'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }

    if proxies is None:
        proxies = {
            'http': 'http://127.0.0.1:10808',
            'https': 'http://127.0.0.1:10808'
        }

    title = None
    progress_url = None

    try:
        params = {
            'format': format,
            'url': url,
        }
        print(f"正在请求下载信息 for: {url}")
        with requests.get(api_url, proxies=proxies, params=params, headers=headers, timeout=10) as r:
            r.raise_for_status()  # 检查HTTP请求是否成功
            data = r.json() # 直接使用 .json() 解析JSON

        title = data.get('title')
        progress_url = data.get('progress_url')

        if not title or not progress_url:
            print(f"错误: 无法从 {url} 获取歌曲标题或进度URL。响应数据: {data}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"请求下载信息时发生网络错误 for {url}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"错误: 无法解析 {api_url} 的JSON响应 for {url}")
        return None
    except Exception as e:
        print(f"获取下载信息时发生未知错误 for {url}: {e}")
        return None

    download_url = None
    for attempt in range(max_retries):
        try:
            print(f"尝试获取下载链接 (尝试 {attempt + 1}/{max_retries}) for: {title}")
            with requests.get(progress_url, headers=headers, proxies=proxies, timeout=10) as r:
                r.raise_for_status()
                progress_data = r.json()

            if progress_data.get('success') == 1:
                download_url = progress_data.get('download_url')
                if download_url:
                    break
                else:
                    print(f"警告: 进度数据中缺少下载URL for {title}. 进度数据: {progress_data}")
            else:
                print(f"进度不成功 for {title}. 进度数据: {progress_data}")

        except requests.exceptions.RequestException as e:
            print(f"获取进度时发生网络错误 for {title}: {e}")
        except json.JSONDecodeError:
            print(f"错误: 无法解析 {progress_url} 的JSON响应 for {title}")
        except Exception as e:
            print(f"获取进度时发生未知错误 for {title}: {e}")

        time.sleep(retry_delay)
    else:
        print(f"错误: 达到最大重试次数，未能获取下载链接 for {title}")
        return None

    if not download_url:
        print(f"错误: 未能获取下载URL for {title}")
        return None

    # 确保下载路径存在
    os.makedirs(download_path, exist_ok=True)
    file_name = f"{title}.mp3"
    full_file_path = os.path.join(download_path, file_name)

    try:
        print(f"正在下载歌曲: {title} 到 {full_file_path}")
        with requests.get(download_url, stream=True, timeout=60) as r:
            r.raise_for_status()
            total_size = int(r.headers.get('content-length', 0))
            with open(full_file_path, 'wb') as f:
                with tqdm(total=total_size, unit='B', unit_scale=True, desc=title, ascii=True) as pbar:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                        pbar.update(len(chunk))
        print(f"\n歌曲下载成功: {full_file_path}")
        return full_file_path
    except requests.exceptions.RequestException as e:
        print(f"下载歌曲时发生网络错误 for {title}: {e}")
        if os.path.exists(full_file_path):
            os.remove(full_file_path) # 下载失败时删除不完整的文件
        return None
    except IOError as e:
        print(f"写入文件时发生IO错误 for {title}: {e}")
        return None
    except Exception as e:
        print(f"下载歌曲时发生未知错误 for {title}: {e}")
        return None


In [ ]:
#api下载
with open('youtube_urls.txt','r') as f:
    song_urls = [line.strip() for line in f.readlines()]
download_path = '/media/ttuubb/1C49-4CEA/laozi'
format = 'mp3'
for url in song_urls:
    download_song(url, download_path=download_path, format=format)
